# Q/A with OpenAI and Trino

## Define Database Connection
This is utilizing an AirBnB data set from the [toolkit use case](https://github.com/msantana09/data-engineering-toolkit/blob/main/UseCase.md)

In [7]:
from langchain.sql_database import SQLDatabase
from sqlalchemy import create_engine

URI = 'trino://trino@localhost:8082/lakehouse/kaggle_airbnb'
OPENAI_MODEL = "gpt-3.5-turbo"

engine = create_engine(URI)
db =SQLDatabase(engine=engine )


/Users/miguel/projects/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'record_count' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/projects/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'file_count' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/projects/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'total_size' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/projects/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_community/utilities/sql_database.py:111: SAWarning: index key 'data' was not located in columns for table 'listings'
  self._metadata.reflect(
/Users/miguel/projects/fun_with_llms/.venv/lib/python3.9/site-packages/langchain_com

### Define LLM and SQL Agent

In [8]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0)
sql_agent = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=False)

### Test it out

In [9]:
response = sql_agent.invoke(
    "Which neighborhood has the best reviews?"
)
print(response['output'])

The neighborhood with the best reviews is Arbor Heights, with an average rating of 98.0.
